# MVP-L: PointPillars 3D 객체 탐지

이 노트북은 PointPillars 모델을 사용하여 포인트 클라우드에서 3D 객체 탐지를 수행합니다.

## 목표
- PointPillars 모델 구현/로드
- 포인트 클라우드 전처리 (voxelization)
- 모델 학습 또는 사전학습 모델 사용
- 3D 바운딩 박스 추론 및 시각화

## 참고
- PointPillars: 포인트 클라우드를 pillar로 변환하여 2D CNN으로 처리
- mmdetection3d를 사용하거나 직접 구현 가능

In [ ]:
# Cell 1: 라이브러리 임포트
import numpy as np
import torch
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

print("PyTorch 버전:", torch.__version__)
print("CUDA 사용 가능:", torch.cuda.is_available())

In [ ]:
# Cell 2: PointPillars 전처리 - Voxelization 및 Pillar 생성
def create_pillars_from_pointcloud(points, voxel_size=[0.16, 0.16, 4], 
                                   point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                                   max_points_per_voxel=100, max_voxels=40000):
    """
    포인트 클라우드를 Pillar로 변환
    
    Args:
        points: (N, 3) 포인트 좌표
        voxel_size: [x, y, z] 방향의 voxel 크기
        point_cloud_range: [x_min, y_min, z_min, x_max, y_max, z_max]
        max_points_per_voxel: 각 voxel당 최대 포인트 수
        max_voxels: 최대 voxel 수
    
    Returns:
        pillars: (P, N, D) 형태의 pillar 데이터
        coords: (P, 3) pillar 좌표
    """
    # 범위 내 포인트만 필터링
    mask = ((points[:, 0] >= point_cloud_range[0]) & 
            (points[:, 0] <= point_cloud_range[3]) &
            (points[:, 1] >= point_cloud_range[1]) & 
            (points[:, 1] <= point_cloud_range[4]) &
            (points[:, 2] >= point_cloud_range[2]) & 
            (points[:, 2] <= point_cloud_range[5]))
    points = points[mask]
    
    # Voxel 좌표 계산
    voxel_coords = ((points[:, :3] - np.array(point_cloud_range[:3])) / 
                   np.array(voxel_size)).astype(np.int32)
    
    # 고유한 voxel 좌표 찾기
    voxel_coords_unique, inverse_indices = np.unique(voxel_coords, axis=0, return_inverse=True)
    
    # 최대 voxel 수로 제한
    if len(voxel_coords_unique) > max_voxels:
        selected_indices = np.random.choice(len(voxel_coords_unique), max_voxels, replace=False)
        voxel_coords_unique = voxel_coords_unique[selected_indices]
        mask = np.isin(inverse_indices, selected_indices)
        inverse_indices = inverse_indices[mask]
        points = points[mask]
        # inverse_indices 재인덱싱
        unique_inverse, idx_map = np.unique(inverse_indices, return_inverse=True)
        inverse_indices = idx_map
    
    # 각 voxel에 대해 포인트 그룹화
    pillars = []
    coords = []
    
    for i, voxel_coord in enumerate(voxel_coords_unique):
        voxel_points = points[inverse_indices == i]
        
        # 최대 포인트 수로 제한 및 패딩
        if len(voxel_points) > max_points_per_voxel:
            voxel_points = voxel_points[:max_points_per_voxel]
        else:
            padding = max_points_per_voxel - len(voxel_points)
            voxel_points = np.pad(voxel_points, ((0, padding), (0, 0)), mode='constant')
        
        pillars.append(voxel_points)
        coords.append(voxel_coord)
    
    return np.array(pillars), np.array(coords)

print("Pillar 생성 함수 정의 완료")

In [ ]:
# Cell 3: 간단한 PointPillars 모델 구조 (참고용)
# 실제 사용 시에는 mmdetection3d의 PointPillars 모델 사용 권장

class SimplePointPillars(nn.Module):
    """
    간단한 PointPillars 모델 구조 (교육용)
    실제 구현은 mmdetection3d 참조
    """
    def __init__(self, num_classes=3):
        super().__init__()
        # Pillar Feature Net (PFE)
        self.pfe = nn.Sequential(
            nn.Linear(9, 64),  # 포인트 특징 차원
            nn.ReLU(),
            nn.Linear(64, 64)
        )
        
        # Backbone (2D CNN)
        self.backbone = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU()
        )
        
        # Detection Head
        self.detection_head = nn.Sequential(
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, num_classes * 7, 1)  # (x, y, z, w, l, h, yaw) * num_classes
        )
    
    def forward(self, pillars, coords):
        # 실제 구현은 더 복잡함
        # 여기서는 구조만 보여줌
        pass

print("모델 구조 정의 완료 (참고용)")

In [ ]:
# Cell 4: mmdetection3d를 사용한 PointPillars 모델 로드 (권장)
# mmdetection3d 설치: 
# !pip install mmcv-full
# !pip install mmdet
# !pip install mmdetection3d

# from mmdet3d.apis import init_model, inference_detector
# 
# config_file = 'pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py'
# checkpoint_file = 'hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth'
# 
# model = init_model(config_file, checkpoint_file, device='cuda:0')
# 
# # 추론
# result, data = inference_detector(model, 'sample.pcd')

print("mmdetection3d 사용 예시 (주석 처리됨)")

In [ ]:
# Cell 5: 3D 바운딩 박스 시각화
def visualize_3d_boxes(points, boxes_3d, ax=None):
    """
    3D 바운딩 박스 시각화
    
    Args:
        points: (N, 3) 포인트 클라우드
        boxes_3d: list of dict with keys 'center', 'size', 'rotation'
        ax: matplotlib 3D axes
    """
    if ax is None:
        fig = plt.figure(figsize=(12, 8))
        ax = fig.add_subplot(111, projection='3d')
    
    # 포인트 클라우드 플롯 (샘플링)
    sample_indices = np.random.choice(len(points), min(10000, len(points)), replace=False)
    ax.scatter(points[sample_indices, 0], 
              points[sample_indices, 1], 
              points[sample_indices, 2], 
              c=points[sample_indices, 2], cmap='jet', s=0.1, alpha=0.5)
    
    # 3D 박스 그리기
    for box in boxes_3d:
        center = box['center']
        size = box['size']  # [w, l, h]
        rotation = box.get('rotation', 0)
        
        # 박스의 8개 꼭짓점 생성
        w, l, h = size
        corners = np.array([
            [-w/2, -l/2, -h/2],
            [w/2, -l/2, -h/2],
            [w/2, l/2, -h/2],
            [-w/2, l/2, -h/2],
            [-w/2, -l/2, h/2],
            [w/2, -l/2, h/2],
            [w/2, l/2, h/2],
            [-w/2, l/2, h/2]
        ])
        
        # 회전 적용
        cos_r = np.cos(rotation)
        sin_r = np.sin(rotation)
        rot_matrix = np.array([[cos_r, -sin_r, 0],
                              [sin_r, cos_r, 0],
                              [0, 0, 1]])
        corners = corners @ rot_matrix.T
        
        # 중심 이동
        corners += center
        
        # 박스 그리기
        edges = [
            [0, 1], [1, 2], [2, 3], [3, 0],  # 하단
            [4, 5], [5, 6], [6, 7], [7, 4],  # 상단
            [0, 4], [1, 5], [2, 6], [3, 7]   # 세로
        ]
        
        for edge in edges:
            ax.plot3D(*corners[edge].T, color='r', linewidth=2)
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title('3D Object Detection Results')
    
    plt.show()
    return ax

print("3D 박스 시각화 함수 정의 완료")

In [ ]:
# Cell 6: KITTI 3D 레이블 파싱
def parse_kitti_3d_label(label_path):
    """
    KITTI 3D 레이블 파싱
    형식: class truncated occluded alpha bbox_2d dimensions_3d location_3d rotation_y
    """
    boxes_3d = []
    if not label_path.exists():
        return boxes_3d
    
    with open(label_path, 'r') as f:
        for line in f:
            if line.strip():
                parts = line.strip().split()
                if len(parts) >= 15:
                    box = {
                        'class': parts[0],
                        'dimensions': [float(parts[8]), float(parts[9]), float(parts[10])],  # h, w, l
                        'location': [float(parts[11]), float(parts[12]), float(parts[13])],  # x, y, z
                        'rotation_y': float(parts[14])
                    }
                    boxes_3d.append(box)
    
    return boxes_3d

# 테스트
project_root = Path.cwd().parent.parent
label_dir = project_root / "dataset" / "training" / "label_2"
if label_dir.exists():
    sample_label = list(label_dir.glob("*.txt"))[0] if list(label_dir.glob("*.txt")) else None
    if sample_label:
        boxes_3d = parse_kitti_3d_label(sample_label)
        print(f"탐지된 3D 객체 수: {len(boxes_3d)}")
        for i, box in enumerate(boxes_3d[:3]):
            print(f"  객체 {i+1}: {box['class']}, 위치: {box['location']}")